<a href="https://colab.research.google.com/github/hailusong/nlp-qa/blob/master/nlp_textsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Text Summarization**
Hyper-params:
TBA

Issues:
1. Max sentence length: 64 (truncated)

## Environment Setup
To get the source code of Huggingface transformer release, do this (assuming the release version is 2.11.0)
```
git fetch -a --tags
git checkout tags/v2.11.0
```

Make sure we have access to file command in Linux

In [0]:
!uname -a
!pip install wget
!apt-get install file

Linux 9af9b4b57846 4.19.104+ #1 SMP Wed Feb 19 05:26:34 PST 2020 x86_64 x86_64 x86_64 GNU/Linux
Reading package lists... Done
Building dependency tree       
Reading state information... Done
file is already the newest version (1:5.32-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [0]:
try:
    import transformers
except ImportError as e:
    # install huggingface
    !pip install transformers

In [0]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

1.5.0+cu101
2.11.0


In [0]:
!ls -al
# !rm -rf nlp-qa
# !git clone --depth 1 https://github.com/hailusong/nlp-qa.git
# !pip install -r nlp-qa/question-answering/requirements.txt

total 272
drwxr-xr-x 1 root root   4096 Jun  7 14:55 .
drwxr-xr-x 1 root root   4096 Jun  7 14:26 ..
drwxr-xr-x 4 root root   4096 Dec 14  2018 cola_public
-rw-r--r-- 1 root root 255330 Jun  7 14:35 cola_public_1.1.zip
drwxr-xr-x 1 root root   4096 Jun  2 16:14 .config
drwxr-xr-x 1 root root   4096 May 29 18:19 sample_data
Cloning into 'nlp-qa'...
fatal: could not read Username for 'https://github.com': No such device or address


## Configuratiuon

In [0]:
EXAMPLE_MODE=True
INLINE_MODE=False

In [0]:
if EXAMPLE_MODE:
  !git clone --depth 1 https://github.com/huggingface/transformers && cd transformers
  !pip install transformers/.
  !pip install nltk py-rouge

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'nlp-qa/question-answering/requirements.txt'


### GPU Setup

In [0]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## Download

### Pre-trained BERTABS fine-tuned on CNN/DM corpus 


In [0]:
if INLINE_MODE:
  # from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  # from transformers import AlbertConfig, AlbertForSequenceClassification
  # import torch
  from modeling_bertabs import BertAbs, build_predictor
  from transformers import BertTokenizer

  tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
  model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
  model.to(args.device)
  model.eval()

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


AlbertConfig {
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 16,
  "num_hidden_groups": 1,
  "num_hidden_layers": 24,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30000
}



Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


In [0]:
if EXAMPLE_MODE:
  !python run_summarization.py \
      --documents_dir $DATA_PATH \
      --summaries_output_dir $SUMMARIES_PATH \ # optional
      --no_cuda false \
      --batch_size 4 \
      --min_length 50 \
      --max_length 200 \
      --beam_size 5 \
      --alpha 0.95 \
      --block_trigram true \


==== Output Layer ====

albert.pooler.weight                                    (1024, 1024)
albert.pooler.bias                                           (1024,)
classifier.weight                                          (2, 1024)
classifier.bias                                                 (2,)


In [0]:
def evaluate(args):

    symbols = {
        "BOS": tokenizer.vocab["[unused0]"],
        "EOS": tokenizer.vocab["[unused1]"],
        "PAD": tokenizer.vocab["[PAD]"],
    }

    if args.compute_rouge:
        reference_summaries = []
        generated_summaries = []

        import rouge
        import nltk

        nltk.download("punkt")
        rouge_evaluator = rouge.Rouge(
            metrics=["rouge-n", "rouge-l"],
            max_n=2,
            limit_length=True,
            length_limit=args.beam_size,
            length_limit_type="words",
            apply_avg=True,
            apply_best=False,
            alpha=0.5,  # Default F1_score
            weight_factor=1.2,
            stemming=True,
        )

    # these (unused) arguments are defined to keep the compatibility
    # with the legacy code and will be deleted in a next iteration.
    args.result_path = ""
    args.temp_dir = ""

    data_iterator = build_data_iterator(args, tokenizer)
    predictor = build_predictor(args, tokenizer, symbols, model)

    logger.info("***** Running evaluation *****")
    logger.info("  Number examples = %d", len(data_iterator.dataset))
    logger.info("  Batch size = %d", args.batch_size)
    logger.info("")
    logger.info("***** Beam Search parameters *****")
    logger.info("  Beam size = %d", args.beam_size)
    logger.info("  Minimum length = %d", args.min_length)
    logger.info("  Maximum length = %d", args.max_length)
    logger.info("  Alpha (length penalty) = %.2f", args.alpha)
    logger.info("  Trigrams %s be blocked", ("will" if args.block_trigram else "will NOT"))

    for batch in tqdm(data_iterator):
        batch_data = predictor.translate_batch(batch)
        translations = predictor.from_batch(batch_data)
        summaries = [format_summary(t) for t in translations]
        save_summaries(summaries, args.summaries_output_dir, batch.document_names)

        if args.compute_rouge:
            reference_summaries += batch.tgt_str
            generated_summaries += summaries

    if args.compute_rouge:
        scores = rouge_evaluator.get_scores(generated_summaries, reference_summaries)
        str_scores = format_rouge_scores(scores)
        save_rouge_scores(str_scores)
        print(str_scores)

## Inference
Source: https://pytorch.org/hub/huggingface_pytorch-transformers/


In [0]:
if QA_EXTRACTION:
  # model = torch.hub.load('huggingface/pytorch-transformers', 'modelForQuestionAnswering', 'bert-large-uncased-whole-word-masking-finetuned-squad')
  # tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-large-uncased-whole-word-masking-finetuned-squad')

  # The format is paragraph first and then question
  # fact = "Jim Henson was a puppeteer"
  # q = "Who was Jim Henson ?"
  fact = input('Fact: ')
  q = input('Question: ')

  # Converts a list of string to a sequence of ids (integer), using the tokenizer and vocabulary. 
  # Also request to add model-specific special tokens (such as beginning of sequence, end of sequence, sequence separator).
  indexed_tokens_fact = tokenizer.encode(fact, add_special_tokens=True)
  indexed_tokens_q = tokenizer.encode(q, add_special_tokens=True)

  # merge fact tokens and Q tokens by removing the START token in Q tokens
  indexed_tokens = indexed_tokens_fact + indexed_tokens_q[1:]
  segments_ids = [0] * len(indexed_tokens_fact) + [1] * len(indexed_tokens_q[1:])

  print(f'Length is {len(indexed_tokens)}, Indexed token: {indexed_tokens}')

  # Predict the start and end positions logits
  segments_tensors = torch.tensor([segments_ids])
  tokens_tensor = torch.tensor([indexed_tokens])

  with torch.no_grad():
      start_logits, end_logits = model(tokens_tensor, token_type_ids=segments_tensors)

  # get the highest prediction
  answer = tokenizer.decode(indexed_tokens[torch.argmax(start_logits):torch.argmax(end_logits)+1])
  print(f'The answer is: {answer}')

  # Or get the total loss which is the sum of the CrossEntropy loss for the start and end token positions (set model to train mode before if used for training)
  # start_positions, end_positions = torch.tensor([12]), torch.tensor([14])
  # multiple_choice_loss = model(tokens_tensor, token_type_ids=segments_tensors, start_positions=start_positions, end_positions=end_positions)